In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from src.L_functions import get_vol, get_neg_sharpe, rast
from src.Models import Multi_CBO_model, gen_Psi,CBO_model
from src.utils import Lmin, avg_x, avg_L, load_config, next_run_index, save_run_config
import random
import os
import sys
from tqdm import trange  # pip install tqdm

In [2]:
def has_converged_all_coords_equal(X: np.ndarray, tol: float = 1e-6) -> bool:
    """
    X: (N, D) 파티클 행렬
    tol: 좌표별 허용 오차. max-min <= tol 이면 같은 값으로 간주.
    """
    # np.ptp = max - min, axis=0 → 각 좌표(l)별 범위
    return np.all(np.ptp(X, axis=0) <= tol)

In [3]:
RESULTS_ROOT = Path("./results")
SIMNAME_SUB  = "250901RAST_Sigma"


# ---- 메인 ----
# np.random.seed(1)
base_cfg   = load_config("configs/configRAST.yaml")
config = dict(base_cfg)
# x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
# np.save( "x0.npy", x0)
# x0 = np.load("x0.npy")

lam1 = 0 
sigma = 0 
for sigma in [0,1,2,3,4,5]:
    print(sigma,lam1)
    run_idx  = next_run_index(RESULTS_ROOT, SIMNAME_SUB)
    run_name = f"{SIMNAME_SUB}_{run_idx}"
    run_root = RESULTS_ROOT / run_name           # 상위 simname 폴더 (여기 아래에 sim0000 ... 생성)
    run_root.mkdir(parents=True, exist_ok=True)

    # 원본 yaml 직접 덮지 말고 메모리에서 수정
    config = dict(base_cfg)
    config["sigma"]    = sigma
    config["simname"] = run_name
    config["lam1"] = lam1
    save_run_config(run_root, config)

    # 문제 정의

    L = rast



    # 공용 초기값 (원하면 실험별 동일 초기값)
    # x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
    # np.save(run_root / "x0.npy", x0)

    nsim    = int(config['sim'])
    maxiter = int(config.get('maxiter', 1000))

    # 진행률: 바깥 루프(시뮬레이션 개수)
    for simnum in range(nsim):
        # 개별 sim 폴더 생성 ...
        sim_dir  = run_root / f"sim{simnum:04d}"
        figs_dir = sim_dir / "figs"
        figs_dir.mkdir(parents=True, exist_ok=True)
    
        model = CBO_model(L, config)
        model.path = str(sim_dir.resolve()) + "/"
        model.make_path()
    
        # x = np.load( "x0.npy")
        x = 2 * np.random.rand(config['nump'], config['D']) + 2
        E = np.zeros(maxiter, dtype=float)
    
        for it in range(maxiter):
    
            if config.get('model', 1) == 1:
                x = model.step(x, L)
            else:
                raise NotImplementedError("model!=1 분기 사용 시 다른 모델 지정 필요")
    
            model.trace_func(x, lambda z: z, "coord")
            model.trace_func(x, lambda x: model.best_loss(x, L), "best_L")
            # --- 진행률 (print 버전) ---
            # if (it + 1) % 50 == 0 or it == maxiter - 1:
            #     pct = (simnum * maxiter + it + 1) / (nsim * maxiter) * 100
            #     print(f"\rProgress: {pct:.1f}%", end="")
            if has_converged_all_coords_equal(x,1e-4):

                break
        # 시뮬레이션 종료 후 개행
        print(it)
        model.save_func("coord", simnum=simnum)
        model.save_func("best_L", simnum=simnum)
        # eps = 1e-12
        # plt.figure()
        # plt.plot(np.arange(maxiter), E + eps, label='E_n')
        # plt.yscale('log')
        # plt.title(f'sim{simnum:04d}')
        # plt.xlabel('iter')
        # plt.ylabel('energy')
        # plt.legend()
        # plt.savefig(figs_dir / "energy.png", dpi=150, bbox_inches="tight")
        # plt.close()


0 0
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
1 0
73
71
81
81
90
102
90
130
96
95
70
114
71
119
97
77
93
87
98
86
96
95
95
90
77
79
82
108
94
83
104
72
92
96
87
75
109
86
86
78
82
102
84
80
92
71
91
83
64
67
2 0
44
55
72
48
67
99
49
46
60
63
54
55
55
45
74
52
82
54
60
62
55
47
62
83
62
57
74
56
42
106
62
79
73
72
111
52
67
49
95
46
63
48
38
71
56
54
96
46
64
94
3 0
68
74
65
103
94
93
66
133
72
92
106
87
108
82
107
66
65
83
91
122
96
100
58
107
93
78
119
119
74
71
72
95
127
51
79
72
85
144
75
65
99
61
77
105
100
61
92
85
84
71
4 0
195
207
242
137
380
310
146
527
187
172
236
175
618
131
164
221
194
358
222
134
271
412
230
338
148
140
161
209
267
308
154
112
191
308
188
159
231
391
221
264
200
242
225
243
157
226
104
183
217
281
5 0
999
999
999
999
999
999
999
999
915
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999


In [4]:
RESULTS_ROOT = Path("./results")
SIMNAME_SUB  = "250901RAST_Lam1"


# ---- 메인 ----
# np.random.seed(1)
base_cfg   = load_config("configs/configRAST.yaml")
config = dict(base_cfg)
# x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
# np.save( "x0.npy", x0)
# x0 = np.load("x0.npy")

lam1 = 0 
sigma = 0 
for lam1 in [0,1,2,3,4,5,6,7,8,9,10]:
    print(sigma,lam1)
    run_idx  = next_run_index(RESULTS_ROOT, SIMNAME_SUB)
    run_name = f"{SIMNAME_SUB}_{run_idx}"
    run_root = RESULTS_ROOT / run_name           # 상위 simname 폴더 (여기 아래에 sim0000 ... 생성)
    run_root.mkdir(parents=True, exist_ok=True)

    # 원본 yaml 직접 덮지 말고 메모리에서 수정
    config = dict(base_cfg)
    config["sigma"]    = sigma
    config["simname"] = run_name
    config["lam1"] = lam1
    save_run_config(run_root, config)

    # 문제 정의

    L = rast



    # 공용 초기값 (원하면 실험별 동일 초기값)
    # x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
    # np.save(run_root / "x0.npy", x0)

    nsim    = int(config['sim'])
    maxiter = int(config.get('maxiter', 1000))

    # 진행률: 바깥 루프(시뮬레이션 개수)
    for simnum in range(nsim):
        # 개별 sim 폴더 생성 ...
        sim_dir  = run_root / f"sim{simnum:04d}"
        figs_dir = sim_dir / "figs"
        figs_dir.mkdir(parents=True, exist_ok=True)
    
        model = CBO_model(L, config)
        model.path = str(sim_dir.resolve()) + "/"
        model.make_path()
    
        # x = np.load( "x0.npy")
        x = 2 * np.random.rand(config['nump'], config['D']) + 2
        E = np.zeros(maxiter, dtype=float)
    
        for it in range(maxiter):
    
            if config.get('model', 1) == 1:
                x = model.step(x, L)
            else:
                raise NotImplementedError("model!=1 분기 사용 시 다른 모델 지정 필요")
    
            model.trace_func(x, lambda z: z, "coord")
            model.trace_func(x, lambda x: model.best_loss(x, L), "best_L")
            # --- 진행률 (print 버전) ---
            # if (it + 1) % 50 == 0 or it == maxiter - 1:
            #     pct = (simnum * maxiter + it + 1) / (nsim * maxiter) * 100
            #     print(f"\rProgress: {pct:.1f}%", end="")
            if has_converged_all_coords_equal(x,1e-4):

                break
        # 시뮬레이션 종료 후 개행
        print(it)
        model.save_func("coord", simnum=simnum)
        model.save_func("best_L", simnum=simnum)
        # eps = 1e-12
        # plt.figure()
        # plt.plot(np.arange(maxiter), E + eps, label='E_n')
        # plt.yscale('log')
        # plt.title(f'sim{simnum:04d}')
        # plt.xlabel('iter')
        # plt.ylabel('energy')
        # plt.legend()
        # plt.savefig(figs_dir / "energy.png", dpi=150, bbox_inches="tight")
        # plt.close()


0 0
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 1
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 2
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 3
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 4
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 5
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
0 6
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93
93


In [1]:
from src.trace_stats import summarize_scalar_trace, summarize_coord_trace, save_stats_json

run_root = "results/250901RAST_Lam1_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_stats = summarize_scalar_trace(run_root, funcname="best_L")
print("best_L mean[-10:]:", bestL_stats["mean"][-10:])
save_stats_json(bestL_stats, f"{run_root}/stats/best_L_stats.json")

# 2) coord (N,D) 통계
coord_stats = summarize_coord_trace(run_root, funcname="coord",
                                    reduce_over="particles_and_sims")
print("coord mean shape:", coord_stats["mean"].shape)  # (T, D)
save_stats_json(coord_stats, f"{run_root}/stats/coord_stats.json")

best_L mean[-10:]: [12.1401077  12.14021899 12.1403196  12.14041051 12.14049262 12.14056676
 12.14063368 12.14069406 12.14074853 12.14079766]
coord mean shape: (94, 15)


In [2]:
run_root = "results/250901RAST_Lam1_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

12.14079765736225
1.287678702455745
94.0
0.0


In [3]:
run_root = "results/250901RAST_Lam1_1"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

9.207177464978404
0.9348316410867692
94.0
0.0


In [4]:
run_root = "results/250901RAST_Lam1_2"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

8.191397200369368
0.8538777953744295
94.0
0.0


In [5]:
run_root = "results/250901RAST_Lam1_3"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

8.065627444332447
0.8452343848327643
94.0
0.0


In [6]:
run_root = "results/250901RAST_Lam1_4"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.8454739430355
0.8112340061489083
94.0
0.0


In [7]:
run_root = "results/250901RAST_Lam1_5"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.432270588587238
0.9892507506199516
94.0
0.0


In [8]:
run_root = "results/250901RAST_Lam1_6"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.382116161340074
1.1121922284175187
94.0
0.0


In [9]:
run_root = "results/250901RAST_Lam1_7"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.465161377864611
1.1025477949382942
94.0
0.0


In [10]:
run_root = "results/250901RAST_Lam1_8"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.124287389124121
1.3452726709904501
94.0
0.0


In [11]:
run_root = "results/250901RAST_Lam1_9"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

6.967587221023652
1.479789648357839
94.0
0.0


In [12]:
run_root = "results/250901RAST_Lam1_10"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

7.002581855284701
1.6287263254347732
94.0
0.0


In [13]:
run_root = "results/250901RAST_Sigma_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

12.157227631268986
1.530580171115343
94.0
0.0


In [14]:
run_root = "results/250901RAST_Sigma_1"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

12.158658970284609
1.269289001734103
89.42
182.2036


In [15]:
run_root = "results/250901RAST_Sigma_2"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

10.840699686963674
1.538156835521
64.52
294.80960000000005


In [16]:
run_root = "results/250901RAST_Sigma_3"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

10.020055574879471
1.6649212982930857
88.84
427.09439999999995


In [17]:
run_root = "results/250901RAST_Sigma_4"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

9.345618312019143
2.0440430247698864
235.74
9925.192400000002


In [18]:
run_root = "results/250901RAST_Sigma_5"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_statsL = summarize_scalar_trace(run_root, funcname="best_L")
print(bestL_statsL['mean'][-1])
print(bestL_statsL['std'][-1])
print(bestL_statsL['iters'])
print(bestL_statsL['iters_var'])

12.888219012676332
1.904789224602043
998.32
138.29760000000005
